In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import cvxopt
from sklearn.metrics import classification_report
from numpy import linalg

In [ ]:
df = pd.read_csv('/content/spambase.data', header=None)
df

In [ ]:
# Calculating NULL percentage in each feature
((df.isnull().sum() / len(df))*100).sort_values(ascending=False).to_frame()

In [ ]:
# Randomize the order of rows in dataset
df = df.reindex(np.random.permutation(df.index)) 
df.head()

In [ ]:
# reset the indices of df
df = df.reset_index(drop=True)

In [ ]:
df.head()

In [ ]:
df[57] = df[57].replace(0,-1)
df[57] = df[57].astype(float)
df[57]

In [ ]:
# Separating X and y
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

# generating splitting indices for 70% separation
split_idx = int(0.7 * len(df))

In [ ]:
# Splitting into train and test dataset
X_train = X[:split_idx]
X_test = X[split_idx:]
y_train = y[:split_idx]
y_test = y[split_idx:]

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
# Standardize the features
mean = np.mean(X_train, axis=0)
std = np.std(X_train, axis=0)

X_train = (X_train - mean) / std

X_test = (X_test - mean) / std

In [ ]:
def linear_kernel(x1, x2):
    return np.dot(x1, x2)

def get_polynomial_kernel(p = 3):
    return lambda x1, x2: (1 + np.dot(x1, x2)) ** p

def get_rbf_kernel(gamma = 0.3):
    return lambda x1, x2: np.exp(-gamma * linalg.norm(x1 - x2) ** 2)

class SVM(object):

    def __init__(self, kernel=linear_kernel, C=None):
        self.kernel = kernel
        self.C = C
        if self.C is not None: self.C = float(self.C)

    def fit(self, X, y):
        n_samples, n_features = X.shape

        # Gram matrix K[i, j] = K(x_i, x_j)
        K = np.zeros((n_samples, n_samples))
        for i in range(n_samples):
            for j in range(n_samples):
                K[i,j] = self.kernel(X[i], X[j])

        # Outer-product gives a matrix such that [i, j] = y_i * y_j
        # Element-wise multiplication gives P, such that:
        # P[i, j] = y_i * y_j * K(x_i, x_j)
        #
        # First factor of objective function:
        # 1/2 x^T P x = 1/2 * sum(alpha_i * alpha_j * y_i * y_j * K(x_i, x_j)
        P = cvxopt.matrix(np.outer(y, y) * K)

        # 2nd factor of objective function:
        # q^T x = -1 * sum(alpha)
        q = cvxopt.matrix(np.ones(n_samples) * -1)

        # Eq constraint (Ax = b): sum(alpha * y) = 0
        A = cvxopt.matrix(y, (1, n_samples))
        b = cvxopt.matrix(0.0)

        # Ineq constraint (Gx <= h):
        # alpha <= C and -alpha <= 0 (equivalent to alpha >= 0)
        if self.C is None:
            G = cvxopt.matrix(np.identity(n_samples) * -1)
            h = cvxopt.matrix(np.zeros(n_samples))
        else:
            tmp1 = np.identity(n_samples) * -1
            tmp2 = np.identity(n_samples)
            G = cvxopt.matrix(np.vstack((tmp1, tmp2)))
            tmp1 = np.zeros(n_samples)
            tmp2 = np.ones(n_samples) * self.C
            h = cvxopt.matrix(np.hstack((tmp1, tmp2)))

        # solve QP problem
        solution = cvxopt.solvers.qp(P, q, G, h, A, b)

        # Lagrange multipliers
        a = np.ravel(solution['x'])

        # Support vectors have non zero lagrange multipliers
        sv = a > 1e-5
        ind = np.arange(len(a))[sv]
        self.a = a[sv]
        self.sv = X[sv]
        self.sv_y = y[sv]
        print(f'{len(self.a)} support vectors out of {n_samples} points')

        # Intercept - average over indices where 0 < alpha_i < C
        self.b = 0
        alpha_diff_C_count = 0
        is_eq_to_C = lambda a: self.C is not None and a > self.C - 1e-5

        for n in range(len(self.a)):
            if is_eq_to_C(self.a[n]):
                continue

            alpha_diff_C_count += 1
            self.b += self.sv_y[n]
            self.b -= np.sum(self.a * self.sv_y * K[ind[n],sv])

        self.b = self.b / alpha_diff_C_count if alpha_diff_C_count > 0 else 0

        # Weight vector: we can't compute w if we are using a kernel
        # since the sum would be over alpha * y * phi(x), where
        # phi is the implicit mapping embedded in the kernel
        if self.kernel == linear_kernel:
            self.w = np.zeros(n_features)
            for n in range(len(self.a)):
                self.w += self.a[n] * self.sv_y[n] * self.sv[n]
        else:
            self.w = None

    def project(self, X):
        if self.w is not None:
            return np.dot(X, self.w) + self.b
        else:
            y_predict = np.zeros(len(X))
            for i in range(len(X)):
                s = 0
                for a, sv_y, sv in zip(self.a, self.sv_y, self.sv):
                    s += a * sv_y * self.kernel(X[i], sv)
                y_predict[i] = s
            return y_predict + self.b

    def predict(self, X):
        return np.sign(self.project(X))

In [ ]:
svm2 = SVM(C=10)
svm2.fit(X_train, y_train)

y_pred2 = []
for x in X_test:
  y_pred2.append(svm2.predict(x))

print(classification_report(y_pred2, y_test))

1 = SPAM

-1 = NOT SPAM